In [ ]:
import tensorflow as tf
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Define dataset paths
dataset_path = "C:/Users/Mr sai kumar/desktop/dataset"
csv_path = os.path.join(dataset_path, "merged_dataset_balanced.csv")
img_size = (128, 128)
batch_size = 32

# ✅ Load dataset
df = pd.read_csv(csv_path)

# ✅ Convert labels to string format
df["label"] = df["label"].astype(str)

# ✅ Check class distribution
print("\n📊 Class Distribution Before Training:\n", df["label"].value_counts())

# ✅ Ensure both classes exist before training
if df["label"].nunique() < 2:
    raise ValueError("Binary classification requires 2 classes (Lesion and Non-Lesion). Found only 1 class!")

# ✅ Data Augmentation
binary_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  # 20% data for validation
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# ✅ Training Data Generator
train_generator = binary_datagen.flow_from_dataframe(
    dataframe=df,
    x_col="image_path",  # Reads images from CSV paths
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training",
    shuffle=True
)

# ✅ Validation Data Generator
val_generator = binary_datagen.flow_from_dataframe(
    dataframe=df,
    x_col="image_path",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)

# ✅ Define CNN Binary Model
binary_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")  # Binary classification output
])

# ✅ Compile the model
binary_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# ✅ Train the model
binary_model.fit(train_generator, validation_data=val_generator, epochs=10)

# ✅ Save the trained model
binary_model.save("C:/Users/Mr sai kumar/desktop/binary_model.keras")

print("\n🎉 Binary Model Trained and Saved Successfully!")



📊 Class Distribution Before Training:
 label
0    46935
1      977
Name: count, dtype: int64
Found 38330 validated image filenames belonging to 2 classes.
Found 9582 validated image filenames belonging to 2 classes.


c:\Users\Mr sai kumar\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Mr sai kumar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 918s 763ms/step - accuracy: 0.9857 - loss: 0.0560 - val_accuracy: 0.9961 - val_loss: 0.0155
Epoch 2/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 1073s 896ms/step - accuracy: 0.9962 - loss: 0.0147 - val_accuracy: 0.9996 - val_loss: 7.4969e-04
Epoch 3/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 802s 670ms/step - accuracy: 0.9974 - loss: 0.0106 - val_accuracy: 0.9998 - val_loss: 6.9319e-04
Epoch 4/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 1003s 837ms/step - accuracy: 0.9973 - loss: 0.0093 - val_accuracy: 0.9995 - val_loss: 0.0017
Epoch 5/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 759s 634ms/step - accuracy: 0.9982 - loss: 0.0083 - val_accuracy: 0.9998 - val_loss: 5.4499e-04
Epoch 6/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 708s 591ms/step - accuracy: 0.9973 - loss: 0.0112 - val_accuracy: 0.9979 - val_loss: 0.0093
Epoch 7/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 771s 643ms/step - accuracy: 0.9972 - loss: 0.0124 - val_accuracy: 0.9977 - val_loss: 0.0099
Epoch 8/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 716s 5

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Set paths
train_dir = "C:/Users/Mr sai kumar/desktop/dataset_split/train"
val_dir = "C:/Users/Mr sai kumar/desktop/dataset_split/   val"

# Image parameters
img_height, img_width = 224, 224
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Simple CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Save best model
checkpoint = ModelCheckpoint("best_multiclass_model.keras", save_best_only=True, monitor='val_accuracy', mode='max')

# Train
history = model.fit(
    train_data,
    epochs=15,
    validation_data=val_data,
    callbacks=[checkpoint]
)


Found 11346 images belonging to 7 classes.
Found 2005 images belonging to 7 classes.


c:\Users\Mr sai kumar\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Mr sai kumar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 316s 874ms/step - accuracy: 0.4753 - loss: 2.1900 - val_accuracy: 0.6668 - val_loss: 1.0509
Epoch 2/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 278s 784ms/step - accuracy: 0.5581 - loss: 1.2323 - val_accuracy: 0.6678 - val_loss: 0.9591
Epoch 3/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 280s 787ms/step - accuracy: 0.6062 - loss: 1.1140 - val_accuracy: 0.6663 - val_loss: 0.9113
Epoch 4/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 264s 743ms/step - accuracy: 0.6395 - loss: 1.0167 - val_accuracy: 0.6753 - val_loss: 0.8976
Epoch 5/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 267s 751ms/step - accuracy: 0.6744 - loss: 0.8886 - val_accuracy: 0.7022 - val_loss: 0.8424
Epoch 6/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 320s 745ms/step - accuracy: 0.7113 - loss: 0.8031 - val_accuracy: 0.6953 - val_loss: 0.8458
Epoch 7/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 271s 762ms/step - accuracy: 0.7502 - loss: 0.6936 - val_accuracy: 0.7007 - val_loss: 0.8581
Epoch 8/15
355/355 ━━━━━━━━━━━━━━━━━━━━ 260s 731ms/step - accuracy: 0.7852 -

In [13]:
model.save("multiclass_model.keras")
print("Multiclass model saved successfully!")


Multiclass model saved successfully!
